Model simulation Strain_2 (Knock-out gdhA).
By knockint-out gdhA we delete the reaction from 2-oxoglutarate to L-glutamate, taking into account that this reaction is reversible in the model, so we want to verify if this knock-out will change the production of Ethylene in our model.

In [31]:
from cobra.io import read_sbml_model
model = read_sbml_model('modified_model.xml')
model

Name,iJO1366
Memory address,255a5de3b50
Number of metabolites,1810
Number of reactions,2592
Number of genes,1368
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


Based on the paper the knock-out that will increase 2-oxoglutarate concentration is gdhA 

In [32]:
# research of gdhA gene in our model
gdhA = None
for gene in model.genes:
    if "gdhA" in gene.name:
        gdhA = gene.id
        break

print("Gene ID for gdhA in model:", gdhA)

Gene ID for gdhA in model: b1761


In [33]:
model.genes.b1761

Gene identifier,b1761
Name,gdhA
Memory address,0x255a6982010
Functional,True
In 1 reaction(s),GLUDy


Let's search for the connected reaction to konck it out

In [34]:
model.reactions.GLUDy

Reaction identifier,GLUDy
Name,Glutamate dehydrogenase (NADP)
Memory address,0x255a716b690
Stoichiometry,glu__L_c + h2o_c + nadp_c <=> akg_c + h_c + nadph_c + nh4_c L-Glutamate + H2O H2O + Nicotinamide adenine dinucleotide phosphate <=> 2-Oxoglutarate + H+ + Nicotinamide adenine dinucleotide phosphate - reduced + Ammonium
GPR,b1761
Lower bound,-1000.0
Upper bound,1000.0


Test the production of EFE reaction before the knock-out

In [35]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9823718127269851 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 12.780573951434883 mmol/gDW*h


Knock-out of the reaction connected to gdhA gene

In [36]:
model.reactions.GLUDy.knock_out()
model.reactions.GLUDy

Reaction identifier,GLUDy
Name,Glutamate dehydrogenase (NADP)
Memory address,0x255a716b690
Stoichiometry,glu__L_c + h2o_c + nadp_c --> akg_c + h_c + nadph_c + nh4_c L-Glutamate + H2O H2O + Nicotinamide adenine dinucleotide phosphate --> 2-Oxoglutarate + H+ + Nicotinamide adenine dinucleotide phosphate - reduced + Ammonium
GPR,b1761
Lower bound,0
Upper bound,0


Maximum theoretical biomass and EFE productivity after the knock-out

In [37]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9487730068947992 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 12.614889336016203 mmol/gDW*h


Combination of the new strain with media implementation

In [38]:
YEPDmedium = model.medium
YEPDmedium['EX_glc__D_e'] = 0
YEPDmedium['EX_sucr_e'] = 40
YEPDmedium['EX_ala__L_e'] = 20
YEPDmedium['EX_arg__L_e'] = 20
YEPDmedium['EX_asn__L_e'] = 20
YEPDmedium['EX_cys__L_e'] = 20
YEPDmedium['EX_gln__L_e'] = 20
YEPDmedium['EX_glu__L_e'] = 20
YEPDmedium['EX_gly_e'] = 0
YEPDmedium['EX_his__L_e'] = 0
YEPDmedium['EX_ile__L_e'] = 0
YEPDmedium['EX_leu__L_e'] = 0
YEPDmedium['EX_met__L_e'] = 0
YEPDmedium['EX_phe__L_e'] = 0
YEPDmedium['EX_pro__L_e'] = 20
YEPDmedium['EX_ser__L_e'] = 20
YEPDmedium['EX_thr__L_e'] = 20
YEPDmedium['EX_trp__L_e'] = 20
YEPDmedium['EX_tyr__L_e'] = 0
YEPDmedium['EX_val__L_e'] = 0
YEPDmedium['EX_lys__L_e'] = 0
YEPDmedium['EX_asp__L_e'] = 20

In [39]:
with model:
    model.medium = YEPDmedium
    solution = model.optimize()
    print('Statistically best (With Sucrose instead of Glucose):')
    print('Maximum theoretical growth-rate:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')
    

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Statistically best (With Sucrose instead of Glucose):
Maximum theoretical growth-rate: 19.191474500961025 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 276.2706896551723 mmol/gDW*h


Maximum theoretical EFE productivity and yield after the knock-out without media implementation

In [40]:
model.objective = model.reactions.eth_tp
eth_production = model.optimize().objective_value
print('Maximum productivity =', eth_production, 'mmol/gDW*h')
   
eth_maximum_yield = eth_production / (-1*(model.reactions.EX_glc__D_e.flux))
print('Maximum theoretical yield =', eth_maximum_yield, 'mmol-eth/mmol-glc')

Maximum productivity = 12.614889336015965 mmol/gDW*h
Maximum theoretical yield = 1.2614889336015964 mmol-eth/mmol-glc
